# 1. Install and Import Dependencies

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

^C


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [73]:
tokens = tokenizer.encode('The room was bad', return_tensors='pt')

In [74]:
result = model(tokens)

In [75]:
result.logits

tensor([[ 2.1702,  1.8589,  0.8291, -1.4760, -2.6991]],
       grad_fn=<AddmmBackward0>)

In [76]:
int(torch.argmax(result.logits))+1

1

# 4. Collect Reviews

In [ ]:
#r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
#soup = BeautifulSoup(r.text, 'html.parser')
#regex = re.compile('.*comment.*')
#results = soup.find_all('p', {'class':regex})
#reviews = [result.text for result in results]}

In [ ]:
reviews

# 5. Load Reviews into DataFrame and Score

In [15]:
import numpy as np
import pandas as pd

In [42]:
df = pd.read_csv('hotel-reviews.csv',nrows=100)


In [43]:
df

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy
...,...,...,...,...,...
95,id10421,"I stayed in this hotel for - nights, and have ...",InternetExplorer,Mobile,happy
96,id10422,The hotel is the closest to the LA convention ...,Firefox,Desktop,not happy
97,id10423,My stay at the W City Center was a Dr. Jekyll-...,Firefox,Mobile,not happy
98,id10424,My wife and I stayed at the Buckingham Hotel f...,IE,Mobile,happy


In [44]:
df['Description'].iloc[0]


"The room was kind of clean but had a VERY strong smell of dogs. Generally below average but ok for a overnight stay if you're not too fussy. Would consider staying again if the price was right. Breakfast was free and just about better than nothing."

In [64]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1    


In [63]:
def to_sentiment(rating):
  rating = int(rating)
  if rating <= 3:
    return 'not happy'
  elif rating == 4:
    return 'happy'
  else:
    return 'happy'

In [58]:
df['Description'].iloc[98]


'My wife and I stayed at the Buckingham Hotel for - nights in mid-April ----. We booked via the hotels website, and got a rate of $--- (plus taxes) for the standard room. it was well-equipped (video player, CD, kitchnettett, safe). We stayed on the --th floor, and our room facing --th Street was a bit noisy, with the - AM garbage truck, ---- police sirens and what have you, so if you are sensitive, you might want to chose a room facing the back. Otherwise the hotel was flawless: the desk downstairs, amenities, cleaning, the bed, everything just perfect (and we are pretty fuzzy). Free wireless in the comfortable business center (but a charge of $-- if you want it in your room). Faboulus value for the location and standard. Highly recommended. And a nice breakfast deli just next doors.'

In [66]:
to_sentiment(sentiment_score(df['Description'].iloc[98]))

'happy'

In [78]:
to_sentiment(sentiment_score('I was told that my room would have several amenities like microwave and a.c. unfortunately There was only a mini fridge in my room and the air conditioner didn’t work at all which is and was a problem in 100⁰ weather.'))

'not happy'

In [ ]:
to_sentiment(sentiment_score('They were extremely accommodating and allowed us to check in early at like 10am. We got to hotel super early and I didn’t wanna wait. So this was a big plus. The sevice was exceptional as well. Would definitely send a friend there.'))

In [68]:
df['sentiment'] = df['Description'].apply(lambda x: to_sentiment(sentiment_score(x[:512])))

In [69]:
df

,User_ID,Description,Browser_Used,Device_Used,Is_Response,sentiment
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy,happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy,not happy
...,...,...,...,...,...,...
95,id10421,"I stayed in this hotel for - nights, and have ...",InternetExplorer,Mobile,happy,happy
96,id10422,The hotel is the closest to the LA convention ...,Firefox,Desktop,not happy,happy
97,id10423,My stay at the W City Center was a Dr. Jekyll-...,Firefox,Mobile,not happy,not happy
98,id10424,My wife and I stayed at the Buckingham Hotel f...,IE,Mobile,happy,happy


In [70]:
if (df['Is_Response'] == df['sentiment']).all():
    print("Les colonnes sont identiques")
else:
    print("Les colonnes sont différentes")

Les colonnes sont différentes


In [71]:
num_diff = (df['Is_Response'] != df['sentiment']).sum()


In [72]:
print("Number of different rows: ", num_diff)


Number of different rows:  17
